In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from snowflake.snowpark.functions import udf
%matplotlib inline
import datetime as dt
import numpy as np
import seaborn as sns

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler
import snowflake.connector
# to save the trained scaler class
import joblib

#Snowflake connection info
# from config import snowflake_conn_prop

import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt



snowflake_conn_prop = {
   "account": "se58322-fsesg",
   "user": "Muhammad Nauman",
   "password": "Muhammad23",
   "role": "DEVELOPER",
   "database": "DL_ESG_DEV",
   "schema": "CUSTOMER",
   "warehouse": "WH_ESG_SUSTAINABILITY",
}

from snowflake.snowpark import version
print(version.VERSION)


connection_paramter = snowflake.connector.connect(account = "se58322-fsesg",
                        user="Muhammad Nauman",
                        password="Muhammad23",
                        role="DEVELOPER",
                        warehouse="WH_ESG_SUSTAINABILITY",
                        ocsp_fail_open=False)

session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

/Users/muhammadnauman/opt/anaconda3/envs/ESG_MVP_IBM/lib/python3.8/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


(1, 0, 0)
[Row(CURRENT_WAREHOUSE()='WH_ESG_SUSTAINABILITY', CURRENT_DATABASE()='DL_ESG_DEV', CURRENT_SCHEMA()='CUSTOMER')]


In [2]:
# 
# qry = "SELECT * FROM DL_ESG_DEV.CUSTOMER.CUSTOMER"
qry = 'SELECT * FROM DL_ESG_DEV.ESG.TRIAL_SCO_ESG_262'

df = pd.read_sql(qry, connection_paramter)

esg_df = df.copy()
# articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
esg_df.head(5)


qry = 'SELECT * FROM DL_ESG_DEV.CUSTOMER.PORTFOLIO'

df_port = pd.read_sql(qry, connection_paramter)

port_df = df_port.copy()
port_df.shape


/var/folders/nr/vyfq21gx5pn1p0025dclx97c0000gn/T/ipykernel_10390/213873629.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry, connection_paramter)
/var/folders/nr/vyfq21gx5pn1p0025dclx97c0000gn/T/ipykernel_10390/213873629.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_port = pd.read_sql(qry, connection_paramter)


(40000, 9)

In [3]:
port_df
esg_df = esg_df.rename({'ticker': 'TICKER'}, axis=1)  # new method

In [19]:
merged_df = port_df.merge(esg_df, on='TICKER', how='left') 

In [20]:
# merged_df = merged_df.head().copy()

In [21]:
merged_df[["esg", "OWNED"]] = merged_df[["esg", "OWNED"]].apply(pd.to_numeric)

In [11]:
merged_df

,R_ID,P_ID,TICKER,BUY_DATE,BUY_VALUE,SHARES_,CURRENCY,OWNED,ADVISOR_ID,date,...,dom_country_iso,dom_region,exch_country_iso,exch_region,economic_sector,industry,esg,esg_e,esg_s,esg_g
0,2983a8d1-d25d-4508-84b2-eb0be516e5be,98b8c46d-386e-491e-aee6-f95386f66b99,TRMB,1457222400000,291,14,JPY,81.333333,f520e014-1243-43b7-a179-0f3ac88d2663,2018-01-01,...,USA,North America,USA,North America,Electronic Technology,Electronic Equipment/Instruments,56.92,47.17,55.37,62.78
1,dca12977-8107-4642-9e2f-f933849e97a0,12453938-522e-4763-9fca-ee51218a4927,0R2I,1627948800000,278,13,USD,16.433333,b2171a34-7c1d-4000-9d47-c63f46e54248,2018-01-01,...,USA,North America,GBR,Europe,Consumer Non-Durables,Apparel/Footwear,43.25,37.66,48.09,43.98
2,f413f981-27ec-4245-bf6b-de3bbe36b3aa,61a5c2b5-ad1e-4d57-9fa7-7b4653362868,0JR2,1604793600000,219,10,JPY,25.266667,0e44ad22-1db9-45c8-a61c-fedfca18e8e9,2018-01-01,...,USA,North America,GBR,Europe,Industrial Services,Oil & Gas Pipelines,51.92,51.40,49.20,53.59
3,4293c697-ed6e-476d-9608-bba687c91fca,4dc5d3b7-1019-4505-92d0-f2c0b0ce9e6c,0I6Q,1648339200000,126,6,USD,8.633333,e97bf078-8afd-4d2f-b4d6-30c239e22359,2018-01-01,...,USA,North America,GBR,Europe,Utilities,Electric Utilities,54.80,65.91,56.83,45.37
4,4484c558-d61b-4e0f-8559-3b1db2d9ad84,377dedb9-2a7a-45d5-a689-a89dc8ccb88e,CTAS,1653264000000,211,10,EUR,6.766667,6975916f-0f6a-405d-9f48-087b89b63b4b,2018-01-01,...,USA,North America,USA,North America,Consumer Services,Other Consumer Services,51.04,57.29,58.64,40.30


In [12]:
merged_df.apply(lambda row: row['OWNED'] * row['esg'], axis=1)

0    4629.493333
1     710.741667
2    1311.845333
3     473.106667
4     345.370667
dtype: float64

In [22]:
merged_df['eventStrength'] = merged_df.apply(lambda row: row['OWNED'] * row['esg'], axis=1)

In [25]:
interactions_df = merged_df.copy()

In [26]:
users_interactions_count_df = interactions_df.groupby(['P_ID', 'TICKER']).size().groupby('P_ID').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['P_ID']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 2000
# users with at least 5 interactions: 2000


In [27]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'P_ID',
               right_on = 'P_ID')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 40000
# of interactions from users with at least 5 interactions: 40000


In [32]:
def smooth_user_preference(x):
    # print(x)
    ## Need to see if this is ok for negatie values
    return math.log(1+abs(x), 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['P_ID', 'TICKER'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 39182


,P_ID,TICKER,eventStrength
0,000da565-ff4d-4d73-bbc1-7905b4582cc1,0HCZ,12.555691
1,000da565-ff4d-4d73-bbc1-7905b4582cc1,0HEU,12.017101
2,000da565-ff4d-4d73-bbc1-7905b4582cc1,0HF7,10.841074
3,000da565-ff4d-4d73-bbc1-7905b4582cc1,0HVP,10.126485
4,000da565-ff4d-4d73-bbc1-7905b4582cc1,0HYE,10.269183
5,000da565-ff4d-4d73-bbc1-7905b4582cc1,0I0J,12.777215
6,000da565-ff4d-4d73-bbc1-7905b4582cc1,0IBC,12.404377
7,000da565-ff4d-4d73-bbc1-7905b4582cc1,0ITV,11.392832
8,000da565-ff4d-4d73-bbc1-7905b4582cc1,0J1R,11.127675
9,000da565-ff4d-4d73-bbc1-7905b4582cc1,0JVI,12.247632


In [34]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['P_ID'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 31345
# interactions on Test set: 7837


In [35]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('P_ID')
interactions_train_indexed_df = interactions_train_df.set_index('P_ID')
interactions_test_indexed_df = interactions_test_df.set_index('P_ID')

In [42]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['TICKER']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['TICKER'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['TICKER']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['TICKER'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['TICKER'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['TICKER'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['TICKER'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_P_ID'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
  

In [43]:
model_evaluator = ModelEvaluator() 

In [44]:
#Computes the most popular items
item_popularity_df = interactions_full_df.groupby('TICKER')['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,TICKER,eventStrength
0,0RR8,1158.008986
1,0IUX,1143.003893
2,0LSZ,1140.894932
3,TFX,1116.716173
4,CTLT,1111.681094
5,0HMZ,1109.759937
6,0L5A,1102.917265
7,AMZ,1098.819110
8,0KOD,1095.308612
9,0R13,1086.696896


In [45]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['TICKER'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
# 	exch_region	economic_sector	industry
            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'TICKER', 
                                                          right_on = 'TICKER')[['eventStrength', 'TICKER', 'esg', 'exch_region', 'economic_sector', 'industry']]


        return recommendations_df
    
popularity_model = PopularityRecommender(item_popularity_df, esg_df)

In [46]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...


TypeError: not all arguments converted during string formatting

In [ ]:
# load_data.py

import pandas as pd
from surprise import Dataset
from surprise import Reader

In [ ]:
# This is the same data that was plotted for similarity earlier
# with one new user "E" who has rated only movie 1
ratings_dict = {
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))

# Loads Pandas dataframe
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)
# Loads the builtin Movielens-100k data
movielens = Dataset.load_builtin('ml-100k')

In [ ]:
# recommender.py

from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)